Pada latihan kali ini kita akan melakukan klasifikasi teks multikelas menggunakan lstm.

Pada latihan ini kita akan menggunakan dataset yang berisi sinopsis dari beberapa film Indonesia dan genrenya. Tujuan kita adalah menentukan genre sebuah film berdasarkan sinopsisnya. Dataset dapat Anda unduh pada [tautan](https://www.kaggle.com/antoniuscs/imdb-synopsis-indonesian-movies) berikut.

Pada cell pertama impor library pandas dan ubah dataset menjadi dataframe. Kemudian buang kolom 'judul_film' karena kita hanya akan menggunakan sinopsis sebagai atribut untuk dilatih pada model.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/imdb_indonesian_movies.csv')
df = df.drop(columns=['judul_film'])

# Panggil fungsi head() pada dataframe untuk menampilkan 5 sampel teratas pada dataset.
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses **one-hot-encoding**. Jalankan kode di bawah untuk melakukan one-hot-encoding dan membuat dataframe baru.

In [ ]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [ ]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

Lalu, bagi data untuk training dan data untuk testing.

In [ ]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer. Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar '16', serta dimensi dari input sebesar nilai `num_words` pada objek tokenizer. Jangan lupa panggil fungsi `compile` dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax'),
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Terakhir kita dapat mulai melatih model kita dengan memanggil fungsi `fit()`.

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 31s - loss: 1.6101 - accuracy: 0.1803 - val_loss: 1.6099 - val_accuracy: 0.1891
Epoch 2/30
26/26 - 27s - loss: 1.6060 - accuracy: 0.2214 - val_loss: 1.6077 - val_accuracy: 0.1990
Epoch 3/30
26/26 - 28s - loss: 1.5643 - accuracy: 0.3396 - val_loss: 1.6142 - val_accuracy: 0.2139
Epoch 4/30
26/26 - 27s - loss: 1.3161 - accuracy: 0.4279 - val_loss: 1.8352 - val_accuracy: 0.2736
Epoch 5/30
26/26 - 28s - loss: 0.9642 - accuracy: 0.5908 - val_loss: 2.4760 - val_accuracy: 0.2289
Epoch 6/30
26/26 - 28s - loss: 0.6068 - accuracy: 0.7861 - val_loss: 2.2634 - val_accuracy: 0.2935
Epoch 7/30
26/26 - 28s - loss: 0.2808 - accuracy: 0.9291 - val_loss: 2.9210 - val_accuracy: 0.3333
Epoch 8/30
26/26 - 28s - loss: 0.1119 - accuracy: 0.9739 - val_loss: 3.4293 - val_accuracy: 0.3333
Epoch 9/30
26/26 - 27s - loss: 0.0505 - accuracy: 0.9900 - val_loss: 3.9139 - val_accuracy: 0.3035
Epoch 10/30
26/26 - 28s - loss: 0.0590 - accuracy: 0.9863 - val_loss: 3.0011 - val_accuracy: 0.3134
Epoch 11/

Akurasi dari model kita menunjukkan terjadinya **overfitting** karena akurasi pada data testing sangat besar, sedangkan akurasi pada data validasi jauh lebih kecil. Hal ini masih sangat luar biasa karena kita hanya memiliki 1000 buah sampel data!

# Selamat! Anda telah paham bagaimana mengembangkan model ML untuk salah satu kasus NLP yaitu klasifikasi teks. Anda telah paham bagaimana memproses teks, serta menggunakan Embedding dan LSTM layer pada arsitektur modul Anda. 

# Setelah submodul ini Anda harus menyelesaikan submission pertama Anda tentang NLP agar bisa mengakses materi-materi selanjutnya.
# Semangat!